# Valence value regression based on Deap Dataset

## 0. This notebook is based on DEAP database

Anyone should refer to DEAP team first

@article{koelstra2012deap,
  title={Deap: A database for emotion analysis; using physiological signals},
  author={Koelstra, Sander and Muhl, Christian and Soleymani, Mohammad and Lee, Jong-Seok and Yazdani, Ashkan and Ebrahimi, Touradj and Pun, Thierry and Nijholt, Anton and Patras, Ioannis},
  journal={IEEE Transactions on Affective Computing},
  volume={3},
  number={1},
  pages={18--31},
  year={2012},
  publisher={IEEE}
}

## 1. Dependency
* numpy
* pyEEG
* sciki-learn

In [1]:
!pip install numpy
!pip install pandas

!pip install scikit-learn


!pip install tensorflow

In [2]:
!pip install git+https://github.com/forrestbao/pyeeg.git


  Cloning https://github.com/forrestbao/pyeeg.git to /tmp/pip-req-build-msu0jfns
  Running command git clone --filter=blob:none --quiet https://github.com/forrestbao/pyeeg.git /tmp/pip-req-build-msu0jfns
  Resolved https://github.com/forrestbao/pyeeg.git to commit a6c18bb093e4748f9d9c208535a6ae024a0802b8
  Preparing metadata (setup.py) ... done


In [3]:
!pip install tensorflow

In [4]:
import numpy as np
import pyeeg as pe
import pickle as pickle
import pandas as pd
import math

from sklearn import svm
from sklearn.preprocessing import StandardScaler 
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split

import os
import tensorflow as tf
import time

2024-08-02 16:08:57.465487: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-08-02 16:08:57.485964: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-02 16:08:57.507219: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-02 16:08:57.514128: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-08-02 16:08:57.531112: I tensorflow/core/platform/cpu_feature_guar

## 2. Global Variables setup
File Name data\SXX.dat, XX \in [0,31]
* data: 40 x 40 x 8064: trial x channel x data
* label: 40 x 4: video/trial x label (valence, arousal, dominance, liking)

Channel Indice: {
* 1 : AF3; 2: F3; 3: F7; 4: FC5; 7: T7; 11: P7; 13: O1
* 17: AF4; 19: F4; 20: F8; 21: FC6; 25: T8; 29: P8; 31: O2 }

In [5]:
eeg_channels = np.array(["Fp1", "AF3", "F3", "F7", "FC5", "FC1", "C3", "T7", "CP5", "CP1", "P3", "P7", "PO3", "O1", "Oz", "Pz", "Fp2", "AF4", "Fz", "F4", "F8", "FC6", "FC2", "Cz", "C4", "T8", "CP6", "CP2", "P4", "P8", "PO4", "O2"])
channel_indices = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31]
peripheral_channels = np.array(["hEOG", "vEOG", "zEMG", "tEMG", "GSR", "Respiration belt", "Plethysmograph", "Temperature"])
band = [4,8,12,30,45] #5 bands
window_size = 256 #Averaging band power of 2 sec
step_size = 16 #Each 0.125 sec update once
sample_rate = 128 #Sampling rate of 128 Hz
subjectList = ['01','02','03','04','05','06','07','08','09','10','11','12','13','14','15','16','17','18','19','20','21','22','23','24','25','26','27','28','29','30','31','32']
#List of subjects

## 3. FFT with pyeeg
* [4-8]: theta band
* [8-12]: alpha band
* [12-30]: beta band 
* [30-45]: low gamma band
* 

In [6]:
from sklearn.decomposition import FastICA
def FFT_Processing (sub, channel_indices, band, window_size, step_size, sample_rate):
    '''
    arguments:  string subject
                list channel indice
                list band
                int window size for FFT
                int step size for FFT
                int sample rate for FFT
    return:     void
    '''
    meta = list([])
    with open('../data/s' + sub + '.dat', 'rb') as file:
        
        subject = pickle.load(file, encoding='latin1') #resolve the python 2 data problem by encoding : latin1

        for i in range (0,40):
            # loop over 0-39 trails
            data = subject["data"][i]
            labels = subject["labels"][i]
            data = data[:32]
            data = np.transpose(data)
            
            start = 0;

            while start + window_size < data.shape[1]:
                meta_array = []
                meta_data = [] #meta vector for analysis
                for j in channel_indices:
                    X = data[j][start : start + window_size] #Slice raw data over 2 sec, at interval of 0.125 sec
                    Y = pe.bin_power(X, band, sample_rate)
                    meta_data.extend(Y[0])  # Append the power values

                meta_array = list([np.array(meta_data), np.array(labels)])  # Create a homogeneous list
                meta.append(meta_array) 

                #meta_array.append(np.array(meta_data))
                #meta_array.append(labels)

                #meta.append(np.array(meta_array))

                start = start + step_size
                
    meta = np.array(meta,dtype=object)
    np.save('psds\s' + sub + str(window_size), meta, allow_pickle=True, fix_imports=True)

def testing (M, L, model):
    '''
    arguments:  M: testing dataset
                L: testing dataset label
                model: scikit-learn model

    return:     void
    '''
    output = model.predict(M[0:78080:32])
    label = L[0:78080:32]

    k = 0
    l = 0

    for i in range(len(label)):
        k = k + (output[i] - label[i])*(output[i] - label[i]) #square difference 

        #a good guess
        if (output[i] > 5 and label[i] > 5):
            l = l + 1
        elif (output[i] < 5 and label[i] <5):
            l = l + 1

    print ("l2 error:", k/len(label), "classification accuracy:", l / len(label),l, len(label))

In [7]:
window_size = 128
step_size = 64
for subjects in subjectList:
    FFT_Processing (subjects, channel_indices, band, window_size, step_size, sample_rate)

In [9]:
window_size = 256
step_size = 16
for subjects in subjectList:
    FFT_Processing (subjects, channel_indices, band, window_size, step_size, sample_rate)


In [7]:
window_size = 640
step_size = 40
for subjects in subjectList:
    FFT_Processing (subjects, channel_indices, band, window_size, step_size, sample_rate)
window_size = 1280
step_size = 80
for subjects in subjectList:
    FFT_Processing (subjects, channel_indices, band, window_size, step_size, sample_rate)

KeyboardInterrupt: 

## 3.Segment of preprocessed data
* training dataset: 80 %
 *testing dataset: 20%

Agrithom pool:
* Support Vector Machine (which kernal?)
* knn
*RF

Best practice could be refered to this paper: 

@article{alarcao2017emotions,
  title={Emotions recognition using EEG signals: A survey},
  author={Alarcao, Soraia M and Fonseca, Manuel J},
  journal={IEEE Transactions on Affective Computing},
  year={2017},
  publisher={IEEE}
}

In [7]:
#for subjects in subjectList:
data_training_2s = []
label_training_2s = []
data_testing_2s = []
label_testing_2s = []
data_all = []
labels_all = []
window_size = 256

for subjects in subjectList:
    with open('psds/psds\s' + subjects + str(window_size) + '.npy', 'rb') as file:
        sub = np.load(file, allow_pickle=True)
        data = [item[0] for item in sub]
        labels = [item[1] for item in sub]
        data_all.extend(data)
        labels_all.extend(labels)

# Convert to numpy arrays for consistency
data_all = np.array(data_all)
labels_all = np.array(labels_all)
        
        # First split: train+val and test (80% train+val, 20% test)
data_training_2s, data_testing_2s, label_training_2s, label_testing_2s = train_test_split(data_all, labels_all, test_size=0.2, random_state=42)
       
       
np.save('out\data_training_2s', np.array(data_training_2s), allow_pickle=True, fix_imports=True)
np.save('out\label_training_2s', np.array(label_training_2s), allow_pickle=True, fix_imports=True)
print("training dataset_2s:", np.array(data_training_2s).shape, np.array(label_training_2s).shape)

np.save('out\data_testing_2s', np.array(data_testing_2s), allow_pickle=True, fix_imports=True)
np.save('out\label_testing_2s', np.array(label_testing_2s), allow_pickle=True, fix_imports=True)
print("testing dataset_2s:", np.array(data_testing_2s).shape, np.array(label_testing_2s).shape)

training dataset_2s: (499712, 128) (499712, 4)
testing dataset_2s: (124928, 128) (124928, 4)


## 4.Regression
### 0. Loading Training and Testing dataset

In [ ]:
with open('out\data_training_2s.npy', 'rb') as fileTrain:
    X_2s  = np.load(fileTrain)
    
with open('out\label_training_2s.npy', 'rb') as fileTrainL:
    Y_2s  = np.load(fileTrainL)

ss = StandardScaler()
X_2s = ss.fit_transform(X_2s)
Z_2s = np.ravel(Y_2s[:, [1]])

Arousal_Train_2s = np.ravel(Y_2s[:, [0]])
Valence_Train_2s = np.ravel(Y_2s[:, [1]])
Domain_Train_2s = np.ravel(Y_2s[:, [2]])
Like_Train_2s = np.ravel(Y_2s[:, [3]])



with open('out\data_testing_2s.npy', 'rb') as fileTrain:
    M_2s  = np.load(fileTrain)
    
with open('out\label_testing_2s.npy', 'rb') as fileTrainL:
    N_2s  = np.load(fileTrainL)

M_2s = ss.transform(M_2s)
L_2s = np.ravel(N_2s[:, [1]])

Arousal_Test_2s = np.ravel(N_2s[:, [0]])
Valence_Test_2s = np.ravel(N_2s[:, [1]])
Domain_Test_2s = np.ravel(N_2s[:, [2]])
Like_Test_2s = np.ravel(N_2s[:, [3]])


### 1. Support Vector Regression
* default setting, l1 error: 1.621761042477756 classification error: 0.6057377049180328 1478 2440

In [ ]:
import numpy as np

# Creating the labels for training data
Y_train_2s = np.zeros_like(Arousal_Train_2s)
condition1_train = (Arousal_Train_2s >= 5) & (Valence_Train_2s >= 5)
condition2_train = (Arousal_Train_2s >= 5) & (Valence_Train_2s < 5)
condition3_train = (Arousal_Train_2s < 5) & (Valence_Train_2s >= 5)
condition4_train = (Arousal_Train_2s < 5) & (Valence_Train_2s < 5)

Y_train_2s[condition1_train] = 0
Y_train_2s[condition2_train] = 1
Y_train_2s[condition3_train] = 2
Y_train_2s[condition4_train] = 3

# Creating the labels for testing data
Y_test_2s = np.zeros_like(Arousal_Test_2s)
condition1_test = (Arousal_Test_2s >= 5) & (Valence_Test_2s >= 5)
condition2_test = (Arousal_Test_2s >= 5) & (Valence_Test_2s < 5)
condition3_test = (Arousal_Test_2s < 5) & (Valence_Test_2s >= 5)
condition4_test = (Arousal_Test_2s < 5) & (Valence_Test_2s < 5)

Y_test_2s[condition1_test] = 0
Y_test_2s[condition2_test] = 1
Y_test_2s[condition3_test] = 2
Y_test_2s[condition4_test] = 3

print("Y_train_2s:", np.unique(Y_train_2s, return_counts=True))
print("Y_test_2s:", np.unique(Y_test_2s, return_counts=True))


In [ ]:
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, f1_score
from sklearn.svm import SVC
from sklearn.metrics import classification_report

param_grid = {
    'C': [0.1, 1, 10],  # Regularization parameter
    'kernel': ['linear', 'rbf']  # Type of kernel
}

# Setup SVM classifier
svm = SVC(probability=True, decision_function_shape='ovo')  # 'ovo' for one-vs-one

# Setup Grid Search for SVM
grid_search = GridSearchCV(
    svm, param_grid, cv=5, scoring='f1_weighted', n_jobs=-1
)

# Fit Grid Search
grid_search.fit(X_2s, Y_train_2s)

# Best parameters
print("Best parameters:", grid_search.best_params_)

# Predict and evaluate
y_pred_2s = grid_search.predict(M_2s)

# Evaluation metrics
accuracy = accuracy_score(Y_test_2s, y_pred_2s)
f1 = f1_score(Y_test_2s, y_pred_2s, average='weighted')

print("Accuracy:", accuracy)
print("F1 score:", f1)

In [ ]:
with open('out\data_training_5s.npy', 'rb') as fileTrain:
    X_5s  = np.load(fileTrain)
    
with open('out\label_training_5s.npy', 'rb') as fileTrainL:
    Y_5s  = np.load(fileTrainL)

ss = StandardScaler()
X_5s = ss.fit_transform(X_5s)
Z_5s = np.ravel(Y_5s[:, [1]])

Arousal_Train_5s = np.ravel(Y_5s[:, [0]])
Valence_Train_5s = np.ravel(Y_5s[:, [1]])
Domain_Train_5s = np.ravel(Y_5s[:, [2]])
Like_Train_5s = np.ravel(Y_5s[:, [3]])



with open('out\data_testing_5s.npy', 'rb') as fileTrain:
    M_5s  = np.load(fileTrain)
    
with open('out\label_testing_5s.npy', 'rb') as fileTrainL:
    N_5s  = np.load(fileTrainL)

M_5s = ss.transform(M_5s)
L_5s = np.ravel(N_5s[:, [1]])

Arousal_Test_5s = np.ravel(N_5s[:, [0]])
Valence_Test_5s = np.ravel(N_5s[:, [1]])
Domain_Test_5s = np.ravel(N_5s[:, [2]])
Like_Test_5s = np.ravel(N_5s[:, [3]])

# Creating the labels for training data
Y_train_5s = np.zeros_like(Arousal_Train_5s)
condition1_train = (Arousal_Train_5s >= 5) & (Valence_Train_5s >= 5)
condition2_train = (Arousal_Train_5s >= 5) & (Valence_Train_5s < 5)
condition3_train = (Arousal_Train_5s < 5) & (Valence_Train_5s >= 5)
condition4_train = (Arousal_Train_5s < 5) & (Valence_Train_5s < 5)

Y_train_5s[condition1_train] = 0
Y_train_5s[condition2_train] = 1
Y_train_5s[condition3_train] = 2
Y_train_5s[condition4_train] = 3

# Creating the labels for testing data
Y_test_5s = np.zeros_like(Arousal_Test_5s)
condition1_test = (Arousal_Test_5s >= 5) & (Valence_Test_5s >= 5)
condition2_test = (Arousal_Test_5s >= 5) & (Valence_Test_5s < 5)
condition3_test = (Arousal_Test_5s < 5) & (Valence_Test_5s >= 5)
condition4_test = (Arousal_Test_5s < 5) & (Valence_Test_5s < 5)

Y_test_5s[condition1_test] = 0
Y_test_5s[condition2_test] = 1
Y_test_5s[condition3_test] = 2
Y_test_5s[condition4_test] = 3

print("Y_train_5s:", np.unique(Y_train_5s, return_counts=True))
print("Y_test_5s:", np.unique(Y_test_5s, return_counts=True))

param_grid = {
    'C': [0.1, 1, 10],  # Regularization parameter
    'kernel': ['linear', 'rbf']  # Type of kernel
}

# Setup SVM classifier
svm = SVC(probability=True, decision_function_shape='ovo')  # 'ovo' for one-vs-one

# Setup Grid Search for SVM
grid_search = GridSearchCV(
    svm, param_grid, cv=5, scoring='f1_weighted', n_jobs=-1
)

# Fit Grid Search
grid_search.fit(X_5s, Y_train_5s)

# Best parameters
print("Best parameters:", grid_search.best_params_)

# Predict and evaluate
y_pred_5s = grid_search.predict(M_5s)

# Evaluation metrics
accuracy = accuracy_score(Y_test_5s, y_pred_5s)
f1 = f1_score(Y_test_5s, y_pred_5s, average='weighted')

print("Accuracy:", accuracy)
print("F1 score:", f1)
